# 리뷰 데이터 클리닝

## 1. 레스토랑 정보, 유저 정보에서 몇가지 정보를 날렸다. 따라서 레스토랑 정보 / 유저정보가 모두 있는 리뷰만 남긴다.

In [2]:
import graphlab as gl
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

This non-commercial license of GraphLab Create for academic use is assigned to yceffort@kaist.ac.kr and will expire on September 07, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1543849384.log
[WARNING] graphlab.deploy._session: Unable to create session in specified location: '/home/yceffort/.graphlab/artifacts'. Using: '/var/tmp/graphlab-yceffort/11769/tmp_session_cd329568-d1d4-407a-9f26-e8128c71e7cc'


In [2]:
user_gf = gl.SFrame('./final-dataset/user-dataset/')
user_gf.head()

average_stars,cool,elite,fans,funny,name,review_count,useful,user_id,yelping_since
2.0,0,None,0,0,Susan,1,0,lzlZwIpuSWXEnNS91wxjHw,2015-09-28
4.0,0,None,0,0,Andy,1,0,QPT4Ud4H5sJVr68yXhoWFw,2016-07-21
4.05,0,None,0,0,Jonathan,19,0,i5YitlHZpf0B3R0s_8NVuw,2014-08-04
3.0,0,None,0,0,Shashank,3,0,s4FoIXE_LSGviTHBe8dmcg,2017-06-18
5.0,0,None,0,0,Lindsay,2,0,h3p6aeVL7vrafSOM50SsCg,2016-02-02
5.0,0,None,0,0,Joshua,1,0,EbJMotYYkq-iq-v1u8wCYA,2013-03-20
2.77,0,None,1,0,Mike,11,8,nnB0AE1Cxp_0154xkhXelw,2014-07-11
4.0,1,None,0,3,Mitch,4,2,XoEnrhtJc2pcdlQ09d8Oug,2013-04-07
5.0,0,None,0,0,Andrea,1,0,QDQTMYp2NocktWN5fHwfIg,2017-03-25
3.86,0,None,0,0,Jessica,6,0,SgYDjNCecPidsRB_su5-tw,2011-01-25


In [3]:
rest_gf = gl.SFrame('./final-dataset/restaurant-dataset/')
rest_gf.head()

address,business_id,categories,city
211 W Monroe St,bFzdJJ3wp3PZssNEsyU23g,"Insurance, FinancialServices ...",Phoenix
"20235 N Cave Creek Rd,Ste 1115 ...",45bWSZtniwPRiqlivpS8Og,"Coffee & Tea, Food",Phoenix
"3417 Derry Road E, Unit103 ...",6OuOZAok8ikONMS_T3EzXg,"Restaurants, Thai",Mississauga
1440 N. Dysart Ave,8-NRKkPY1UiFXW20WXKiXg,"Mexican, Restaurants",Avondale
209 Oakland Ave,UTm5QZThPQlT35mkAcGOjg,"Flowers & Gifts, GiftShops, Shopping ...",Pittsburgh
2801 N 15th Ave,_c3ixq9jYKxhLUB0czi0ug,"Bars, Sports Bars, DiveBars, Burgers, Nightl ...",Phoenix
4216 Saint-Laurent Boul,bBUMib8l6Me1ZB1_Qkezkg,"Restaurants, Pakistani,Indian, Middle Eastern ...",Montréal
703 N Rancho Dr,hTzcHtk4-0QJnFUbkKpd5Q,"Shopping, Fashion,Department Stores ...",Las Vegas
1549 N Rancho Dr,UwIpS9UKsPiKAv1fiEYhqg,"Financial Services, CheckCashing/Pay-day Loans, ...",Las Vegas
,YZCHr68c5aEVHz0bkq9K2g,"Home Services,Masonry/Concrete, ...",Las Vegas


In [4]:
review_gf = gl.SFrame('./dataset/yelp_academic_dataset_review/')
review_gf.head()

business_id,cool,date,funny,review_id,stars,text,useful
iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Notthe best I've had. I ...,0
pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! Myfiance And I go here ...,0
jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,LZp4UX5zK3e-c5ZGSeo3kA,1,Terrible. Dry corn bread.Rib tips were all fat ...,3
elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,Er4NBWCmCD4nM8_p1GRdow,2,Back in 2005-2007 thisplace was my FAVORITE ...,2
Ums3gaP2qM3W1XcA5r6SsQ,0,2014-09-05,0,jsDu6QEJHbwP2Blom1PLCA,5,Delicious healthy food.The steak is amazing. ...,0
vgfcTvK81oD4r50NMjU2Ag,0,2011-02-25,0,pfavA0hr3nyqO61oupj-lA,1,This place sucks. Thecustomer service is ...,2
AxeQEz3-s9_1TyIo-G7UQw,0,2011-10-10,0,brokEno2n7s4vrwmmUdr9w,5,"If you like Thai food,you have to try the ...",1
zdE82PiD6wquvjYLyhOJNA,0,2012-04-18,1,kUZWBVZvhWuC8TWUg5AYyA,5,AMAZING!!!\n\n I wasreferred here by a fr ...,0
EAwh1OmG6t6p3nRaZOW_AA,0,2011-02-25,0,wcqt0III88LEcm19IxFFyA,4,Ribs = amazing\n2 hourwait time= not so ...,0
atVh8viqTj-sqDJ35tAYVg,1,2012-11-09,2,LWUtqzNthMM3vpWZIFBlPw,2,"Food is pretty good, notgonna lie. BUT you have ...",1


In [5]:
rest_ids = rest_gf['business_id']
rest_ids

dtype: str
Rows: 149518
['bFzdJJ3wp3PZssNEsyU23g', '45bWSZtniwPRiqlivpS8Og', '6OuOZAok8ikONMS_T3EzXg', '8-NRKkPY1UiFXW20WXKiXg', 'UTm5QZThPQlT35mkAcGOjg', '_c3ixq9jYKxhLUB0czi0ug', 'bBUMib8l6Me1ZB1_Qkezkg', 'hTzcHtk4-0QJnFUbkKpd5Q', 'UwIpS9UKsPiKAv1fiEYhqg', 'YZCHr68c5aEVHz0bkq9K2g', 'gJ5xSt6147gkcZ9Es0WxlA', '3ByGQOVgds2YEu6kzl-XEQ', '5J3b7j3Fzo9ISjChmoUoUA', 'CeuTRtwsq6w5rztGOyNMPg', 'guRyrKMBDkcHExGzwxSfvg', '65DAbjsiws2_T600pNpWFQ', '6YC6CsXRrmPv_iwfvc9onA', '42uRhhQDzbtKrUYoBPXS8A', 'B_mz59xi-Q3zCIiXsnBM_A', 'kgffcoxT6BQp-gJ-UQ7Czw', '0jtRI7hVMpQHpUVtUy4ITw', 'pbluh-ZZi_i8XlprfWB1nQ', 'UeyHx5LMFjjI4d-NTYkX9Q', 'qHF2s8U2yzPLgIOGH_IMBw', 'iTZDPTc36guXhNhoK-jwmw', 'AcGRSWCpb7YB95MTsHlGEw', 'Zkesf4VRamyOBswJA4_ZhQ', '7A6mfqSfG3AG3WZcr45m-Q', '3pcHGNI8WHv44fcvgkLrMg', 'zHwXoh40k86P0aiN1aix9Q', '0i6GlR5YTCMlwthjqw6ZTQ', 'oSS0p8TeF9BTTD7QWtO6gg', '2C8x0RxmqvCOmhvXce4-HA', 'cKRMmytHxaSt8F0SMEzKqg', '5vpMTDaAkpu9PBW1psG_RA', 'DR30lzIHVTF6xhyMI-3IlQ', 'wv9KN5x8L2qzKFq_6Hzf9g', 'VVtPNiqwtR_O

In [6]:
review_gf = review_gf.filter_by(rest_ids, 'business_id')
review_gf.head()

business_id,cool,date,funny,review_id,stars,text,useful
iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Notthe best I've had. I ...,0
pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! Myfiance And I go here ...,0
jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,LZp4UX5zK3e-c5ZGSeo3kA,1,Terrible. Dry corn bread.Rib tips were all fat ...,3
elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,Er4NBWCmCD4nM8_p1GRdow,2,Back in 2005-2007 thisplace was my FAVORITE ...,2
zdE82PiD6wquvjYLyhOJNA,0,2012-04-18,1,kUZWBVZvhWuC8TWUg5AYyA,5,AMAZING!!!\n\n I wasreferred here by a fr ...,0
EAwh1OmG6t6p3nRaZOW_AA,0,2011-02-25,0,wcqt0III88LEcm19IxFFyA,4,Ribs = amazing\n2 hourwait time= not so ...,0
atVh8viqTj-sqDJ35tAYVg,1,2012-11-09,2,LWUtqzNthMM3vpWZIFBlPw,2,"Food is pretty good, notgonna lie. BUT you have ...",1
yFumR3CWzpfvTH2FCthvVw,0,2016-06-15,0,STiFMww2z31siPY7BWNC2g,5,I have been an EmeraldClub member for a number ...,0
hdgYnadxg0GANhWOJabr2g,0,2014-08-23,0,RgqWdZA4xR023iP3T6jVfA,5,I went there twice and Iam pretty happy with my ...,0
yEOu75XjwczngvWWlr0M_A,0,2016-02-17,0,-STQDcMcBcWX0H_NrsfR2Q,4,"Great for the price, only$25 for long hair! Was ...",1


In [7]:
len(review_gf)

5168168

In [8]:
review_gf = review_gf.filter_by(user_gf['user_id'], 'user_id')
len(review_gf)

4797950

In [9]:
len(user_gf)

1407452

## 2. 데이터 저장

In [10]:
review_gf.save('./final-dataset/review-dataset/')

In [12]:
df = gl.SFrame('./final-dataset/review-dataset/')
df.num_rows()

4797950

## 3. 새로 저장한 레스토랑 정보 기반으로 다시 한번 필터링한다.

In [3]:
rf = gl.SFrame('./final-dataset/restaurant-dataset-v2/')
rf.head(5)

Columns:
	id	int
	address	str
	business_id	str
	city	str
	longitude	float
	latitude	float
	name	str
	postal_code	str
	review_count	int
	stars	float
	state	str
	delivery_yn	int
	card_yn	int
	takeout_yn	int
	alcohol_yn	int
	price_category	float
	created_at	datetime
	updated_at	datetime

Rows: 5

Data:
+----+-------------------------------+------------------------+-------------+
| id |            address            |      business_id       |     city    |
+----+-------------------------------+------------------------+-------------+
| 2  | 20235 N Cave Creek Rd, Ste... | 45bWSZtniwPRiqlivpS8Og |   Phoenix   |
| 3  |  3417 Derry Road E, Unit 103  | 6OuOZAok8ikONMS_T3EzXg | Mississauga |
| 4  |       1440 N. Dysart Ave      | 8-NRKkPY1UiFXW20WXKiXg |   Avondale  |
| 6  |        2801 N 15th Ave        | _c3ixq9jYKxhLUB0czi0ug |   Phoenix   |
| 7  |    4216 Saint-Laurent Boul    | bBUMib8l6Me1ZB1_Qkezkg |   Montréal  |
+----+-------------------------------+------------------------+-------------+
+-----------+----------+----------------------------+-------------+--------------+
| longitude | latitude |            name            | postal_code | review_count |
+-----------+----------+----------------------------+-------------+--------------+
|  -112.03  | 33.6714  | The Coffee Bean & Tea Leaf |    85024    |      63      |
|  -79.6328 | 43.7129  |        Thai One On         |   L4T 1A8   |      7       |
|  -112.341 | 33.4481  |  Filiberto's Mexican Food  |    85323    |      40      |
|  -112.091 | 33.4798  |  Original Hamburger Works  |    85007    |     277      |
|  -73.5822 |  45.518  |   Mysore Indian Cuisine    |   H2W 1Z3   |      19      |
+-----------+----------+----------------------------+-------------+--------------+
+-------+-------+-------------+---------+------------+------------+----------------+
| stars | state | delivery_yn | card_yn | takeout_yn | alcohol_yn | price_category |
+-------+-------+-------------+---------+------------+------------+----------------+
|  4.0  |   AZ  |      0      |    1    |     1      |     1      |      1.0       |
|  2.0  |   ON  |      0      |    1    |     1      |     0      |      2.0       |
|  2.5  |   AZ  |      0      |    1    |     1      |     0      |      1.0       |
|  4.0  |   AZ  |      0      |    1    |     1      |     1      |      1.0       |
|  3.5  |   QC  |      1      |    1    |     1      |     1      |      2.0       |
+-------+-------+-------------+---------+------------+------------+----------------+
+---------------------------+---------------------------+
|         created_at        |         updated_at        |
+---------------------------+---------------------------+
| 2018-11-22 02:40:21+00:00 | 2018-11-22 02:40:21+00:00 |
| 2018-11-22 02:40:21+00:00 | 2018-11-22 02:40:21+00:00 |
| 2018-11-22 02:40:21+00:00 | 2018-11-22 02:40:21+00:00 |
| 2018-11-22 02:40:21+00:00 | 2018-11-22 02:40:21+00:00 |
| 2018-11-22 02:40:21+00:00 | 2018-11-22 02:40:21+00:00 |
+---------------------------+---------------------------+
[5 rows x 18 columns]

In [5]:
reviews = gl.SFrame('./final-dataset/review-dataset/')
reviews.head(5)

Columns:
	business_id	str
	cool	int
	date	str
	funny	int
	review_id	str
	stars	int
	text	str
	useful	int
	user_id	str

Rows: 5

Data:
+------------------------+------+------------+-------+------------------------+
|      business_id       | cool |    date    | funny |       review_id        |
+------------------------+------+------------+-------+------------------------+
| iCQpiavjjPzJ5_3gPD5Ebg |  0   | 2011-02-25 |   0   | x7mDIiDB3jEiPGPHOmDzyw |
| pomGBqfbxcqPv14c3XH-ZQ |  0   | 2012-11-13 |   0   | dDl8zu1vWPdKGihJrwQbpw |
| jtQARsP6P-LbkyjbO1qNGg |  1   | 2014-10-23 |   1   | LZp4UX5zK3e-c5ZGSeo3kA |
| elqbBhBfElMNSrjFqW3now |  0   | 2011-02-25 |   0   | Er4NBWCmCD4nM8_p1GRdow |
| zdE82PiD6wquvjYLyhOJNA |  0   | 2012-04-18 |   1   | kUZWBVZvhWuC8TWUg5AYyA |
+------------------------+------+------------+-------+------------------------+
+-------+-------------------------------+--------+------------------------+
| stars |              text             | useful |        user_id         |
+-------+-------------------------------+--------+------------------------+
|   2   | The pizza was okay. Not th... |   0    | msQe1u7Z_XuqjGoqhB0J5g |
|   5   | I love this place! My fian... |   0    | msQe1u7Z_XuqjGoqhB0J5g |
|   1   | Terrible. Dry corn bread. ... |   3    | msQe1u7Z_XuqjGoqhB0J5g |
|   2   | Back in 2005-2007 this pla... |   2    | msQe1u7Z_XuqjGoqhB0J5g |
|   5   | AMAZING!!!\n\n I was refer... |   0    | msQe1u7Z_XuqjGoqhB0J5g |
+-------+-------------------------------+--------+------------------------+
[5 rows x 9 columns]

In [8]:
filtered_reviews = reviews.filter_by(rf['business_id'].unique(), 'business_id')
len(filtered_reviews), len(reviews)

(3365252, 4797950)

In [9]:
filtered_reviews.save('./final-dataset/review-dataset-v2/')

In [10]:
filtered_reviews.export_csv('./final-dataset/review-dataset-v2.csv')

In [11]:
test = gl.SFrame.read_csv('./final-dataset/review-dataset-v2.csv')
test.head(5)

Finished parsing file /home/yceffort/2018_itm/final-dataset/review-dataset-v2.csv

Parsing completed. Parsed 100 lines in 1.43798 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,str,int,str,int,str,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 85762 lines. Lines per second: 55636.4

Read 677808 lines. Lines per second: 92687.1

...

Read 3183961 lines. Lines per second: 79533.6

Read 3334040 lines. Lines per second: 67910.1

Finished parsing file /home/yceffort/2018_itm/final-dataset/review-dataset-v2.csv

Parsing completed. Parsed 3365252 lines in 49.4065 secs.

Columns:
	business_id	str
	cool	int
	date	str
	funny	int
	review_id	str
	stars	int
	text	str
	useful	int
	user_id	str

Rows: 5

Data:
+------------------------+------+------------+-------+------------------------+
|      business_id       | cool |    date    | funny |       review_id        |
+------------------------+------+------------+-------+------------------------+
| iCQpiavjjPzJ5_3gPD5Ebg |  0   | 2011-02-25 |   0   | x7mDIiDB3jEiPGPHOmDzyw |
| pomGBqfbxcqPv14c3XH-ZQ |  0   | 2012-11-13 |   0   | dDl8zu1vWPdKGihJrwQbpw |
| jtQARsP6P-LbkyjbO1qNGg |  1   | 2014-10-23 |   1   | LZp4UX5zK3e-c5ZGSeo3kA |
| elqbBhBfElMNSrjFqW3now |  0   | 2011-02-25 |   0   | Er4NBWCmCD4nM8_p1GRdow |
| zdE82PiD6wquvjYLyhOJNA |  0   | 2012-04-18 |   1   | kUZWBVZvhWuC8TWUg5AYyA |
+------------------------+------+------------+-------+------------------------+
+-------+-------------------------------+--------+------------------------+
| stars |              text             | useful |        user_id         |
+-------+-------------------------------+--------+------------------------+
|   2   | The pizza was okay. Not th... |   0    | msQe1u7Z_XuqjGoqhB0J5g |
|   5   | I love this place! My fian... |   0    | msQe1u7Z_XuqjGoqhB0J5g |
|   1   | Terrible. Dry corn bread. ... |   3    | msQe1u7Z_XuqjGoqhB0J5g |
|   2   | Back in 2005-2007 this pla... |   2    | msQe1u7Z_XuqjGoqhB0J5g |
|   5   | AMAZING!!!\n\n I was refer... |   0    | msQe1u7Z_XuqjGoqhB0J5g |
+-------+-------------------------------+--------+------------------------+
[5 rows x 9 columns]

## 4. 라스베거스만 한다.

In [3]:
rt = gl.SFrame('./las_vegas_v2/restaurant/')

In [4]:
review = gl.SFrame('./final-dataset/review-dataset-v2/')

In [5]:
lv_review = review.filter_by(rt['business_id'].unique(), 'business_id')
lv_review.head()

business_id,cool,date,funny,review_id,stars,text,useful
iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Notthe best I've had. I ...,0
pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,dDl8zu1vWPdKGihJrwQbpw,5,I love this place! Myfiance And I go here ...,0
jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,LZp4UX5zK3e-c5ZGSeo3kA,1,Terrible. Dry corn bread.Rib tips were all fat ...,3
elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,Er4NBWCmCD4nM8_p1GRdow,2,Back in 2005-2007 thisplace was my FAVORITE ...,2
zdE82PiD6wquvjYLyhOJNA,0,2012-04-18,1,kUZWBVZvhWuC8TWUg5AYyA,5,AMAZING!!!\n\n I wasreferred here by a fr ...,0
EAwh1OmG6t6p3nRaZOW_AA,0,2011-02-25,0,wcqt0III88LEcm19IxFFyA,4,Ribs = amazing\n2 hourwait time= not so ...,0
atVh8viqTj-sqDJ35tAYVg,1,2012-11-09,2,LWUtqzNthMM3vpWZIFBlPw,2,"Food is pretty good, notgonna lie. BUT you have ...",1
faPVqws-x-5k2CQKDNtHxw,0,2015-03-25,2,QHueaxKHuBpbDOkWb0Cyfw,1,People please...the onlygood thing about this ...,2
ruWTngdiC68O9la27hvvHw,0,2016-02-24,0,Tifm4ETlt4FXiuGwhq__lg,4,This place is not just arestaurant has a great ...,1
7KkgMcbVaetryW1wwpzvvA,0,2016-02-29,0,htV6dNmUqaXwCIRt7vfO5A,4,It great fast food andget large portions. It ...,0


In [6]:
len(lv_review), len(review)

(889957, 3365252)

In [7]:
lv_review.save('./las_vegas_v2/reviews')
lv_review.export_csv('./las_vegas_v2/reviews.csv')

In [8]:
to_rt = gl.SFrame('./toronto/restaurant')

In [9]:
to_review = review.filter_by(to_rt['business_id'].unique(), 'business_id')
to_review.head()

business_id,cool,date,funny,review_id,stars,text,useful
f5O7v_X_jCg2itqacRfxhg,0,2017-10-12,0,kS4hrhEScwB9V5JATYjvVQ,5,Sansotei serves some topnotch ramen. They tak ...,0
7xA6iSP0Ndn08tpBFQtUKA,0,2017-05-22,0,YDJDfKnx6VpMMo4EBxycGg,1,Non-existened service!The waiter did not even ...,0
aAYl1-tsvP9Kzs7YESi8Rg,0,2015-10-04,0,1_ntTyrsYYYAy41nm3UD3Q,2,Extremelydisappointing.\n\nThis ...,0
_zTW9DhfQxU7HaVAu-Tsmg,0,2014-12-16,2,mIyMereUwhex_nDr63YOAg,1,"Went in looking for agood knife, as an Xmas ...",3
ELt26I6N0VqVywR5J4TMlw,1,2014-12-20,0,Swh8eQtbNdcuMyHP5OXkfQ,4,Been coming here for thelunch special every ...,2
ALkOqlWODyxPg5VSp0HLrg,0,2013-04-08,1,Ku0bs-MtWERfUmgw2jchMA,5,Super solid. Arrived at10ish on a Saturday ...,0
3VvPKmr-6LXJ7aWlFtp4Yg,0,2013-05-20,0,zPRRy6tQZQEAP46IGQn_8Q,4,I've been waiting forLive Market to open f ...,2
M-UEi9A08B4URLfrsql3BA,0,2012-04-25,2,GBDHc0_hT12_PvM8NONBaQ,1,"Their ""Vegan Delight""wrap has Havarti cheese ...",1
nhfZW6Pi-oLQuhYcrcZJWw,4,2010-10-05,0,sQlDlTNl8qSciIUQOzFdUw,5,I ordered a soy latte andit was PERFECT. Just a ...,2
dgpPVSFL84ONx477Lln2sw,3,2011-01-12,0,H9BpJnQ9Zy90Ib6qGA3DZw,4,The first time Merchantsof Green coffee was ...,2


In [10]:
to_review.save('./toronto/reviews')
to_review.export_csv('./toronto/reviews.csv')

In [11]:
len(to_review)

284245

In [12]:
len(lv_review)

889957